Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [8]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from time import time

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [22]:
batch_size = 128
l2_weight =1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
  biases_hidden = tf.Variable(tf.zeros([1024]))
  
  # Training computation.
  hidden_2 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden)+biases_hidden)
  logits = tf.matmul(hidden_2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_weight*tf.nn.l2_loss(weights_hidden)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+biases_hidden)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_valid, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(hidden_test, weights) + biases)

In [23]:
num_steps = 3001
t0 = time()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("##################################")
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("time taken: %0.3fs"%(time()-t0))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
##################################
Minibatch loss at step 0: 652.904663
Minibatch accuracy: 7.8%
Validation accuracy: 25.0%
time taken: 1.342s
##################################
Minibatch loss at step 500: 199.693939
Minibatch accuracy: 83.6%
Validation accuracy: 79.0%
time taken: 15.323s
##################################
Minibatch loss at step 1000: 117.230629
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
time taken: 31.502s
##################################
Minibatch loss at step 1500: 68.586311
Minibatch accuracy: 88.3%
Validation accuracy: 82.2%
time taken: 49.391s
##################################
Minibatch loss at step 2000: 41.305538
Minibatch accuracy: 90.6%
Validation accuracy: 84.4%
time taken: 66.113s
##################################
Minibatch loss at step 2500: 25.108912
Minibatch accuracy: 90.6%
Validation accuracy: 85.5%
time taken: 82.209s
##################################
Minibatch loss at step 3000: 15.405045
Minibatch accuracy: 85.9%
Validatio

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [47]:
batch_size = 128
l2_weight =0.0

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
  biases_hidden = tf.Variable(tf.zeros([1024]))
  
  # Training computation.
  hidden_2 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden)+biases_hidden)
  logits = tf.matmul(hidden_2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_weight*tf.nn.l2_loss(weights_hidden)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+biases_hidden)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_valid, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(hidden_test, weights) + biases)

In [48]:
num_steps = 101
t0 = time()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step%3) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("##################################")
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("time taken: %0.3fs"%(time()-t0))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
##################################
Minibatch loss at step 0: 277.218231
Minibatch accuracy: 8.6%
Validation accuracy: 29.5%
time taken: 1.214s
##################################
Minibatch loss at step 10: 41.112579
Minibatch accuracy: 86.7%
Validation accuracy: 72.4%
time taken: 2.029s
##################################
Minibatch loss at step 20: 9.286970
Minibatch accuracy: 97.7%
Validation accuracy: 73.7%
time taken: 2.733s
##################################
Minibatch loss at step 30: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
time taken: 3.469s
##################################
Minibatch loss at step 40: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
time taken: 4.273s
##################################
Minibatch loss at step 50: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
time taken: 5.074s
##################################
Minibatch loss at step 60: 0.000000
Minibatch accuracy: 100.0%
Validation accuracy: 74.0%
t

minibatch accuracy hits 100% very quickly but validation and test accuracy suffers as a result

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [65]:
batch_size = 128
l2_weight =0.0


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #placeholders
  keep_prob = tf.placeholder(tf.float32)

  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,1024]))
  biases_hidden = tf.Variable(tf.zeros([1024]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden)+biases_hidden)
  hidden_dropout = tf.nn.dropout(hidden,keep_prob)
  logits = tf.matmul(hidden_dropout, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_weight*tf.nn.l2_loss(weights_hidden)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+biases_hidden)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_valid, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+biases_hidden)
  test_prediction = tf.nn.softmax(
    tf.matmul(hidden_test, weights) + biases)

In [66]:
num_steps = 101
t0 = time()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = ((step%3) * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob:0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 10 == 0):
      print("##################################")
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("time taken: %0.3fs"%(time()-t0))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
##################################
Minibatch loss at step 0: 520.452820
Minibatch accuracy: 10.9%
Validation accuracy: 21.5%
time taken: 1.181s
##################################
Minibatch loss at step 10: 148.053986
Minibatch accuracy: 73.4%
Validation accuracy: 67.3%
time taken: 1.927s
##################################
Minibatch loss at step 20: 4.803474
Minibatch accuracy: 96.9%
Validation accuracy: 75.5%
time taken: 2.669s
##################################
Minibatch loss at step 30: 11.416954
Minibatch accuracy: 95.3%
Validation accuracy: 75.0%
time taken: 3.530s
##################################
Minibatch loss at step 40: 2.593599
Minibatch accuracy: 96.1%
Validation accuracy: 76.3%
time taken: 4.685s
##################################
Minibatch loss at step 50: 4.729964
Minibatch accuracy: 97.7%
Validation accuracy: 75.0%
time taken: 5.816s
##################################
Minibatch loss at step 60: 1.549042
Minibatch accuracy: 98.4%
Validation accuracy: 76.0%
ti

dropout helps with overfitting to minibatch providing better generality

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [107]:
batch_size = 128
l2_weight =0.0


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #placeholders
  keep_prob = tf.placeholder(tf.float32)

  #hidden 1
  weights_hidden_1 = tf.Variable(tf.truncated_normal([image_size*image_size,256]))
  biases_hidden_1 = tf.Variable(tf.zeros([256]))

  #hidden 2
  weights_hidden_2 = tf.Variable(tf.truncated_normal([256,256]))
  biases_hidden_2 = tf.Variable(tf.zeros([256]))

  # final regression
  weights = tf.Variable(tf.truncated_normal([256, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
    
  # Training computation.
  hidden_1 = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden_1)+biases_hidden_1)
  hidden_dropout_1 = tf.nn.dropout(hidden_1,keep_prob)

  hidden_2 = tf.nn.relu(tf.matmul(hidden_1,weights_hidden_2)+biases_hidden_2)
  hidden_dropout_2 = tf.nn.dropout(hidden_2,keep_prob)

  logits = tf.matmul(hidden_2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_weight*tf.nn.l2_loss(weights_hidden_2)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.01, global_step, 1000,0.65)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  hidden_valid_1 = tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden_1)+biases_hidden_1)
  hidden_valid_2 = tf.nn.relu(tf.matmul(hidden_valid_1,weights_hidden_2)+biases_hidden_2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(hidden_valid_2, weights) + biases)
  hidden_test_1 = tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden_1)+biases_hidden_1)
  hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test_1,weights_hidden_2)+biases_hidden_2)
  test_prediction = tf.nn.softmax(
    tf.matmul(hidden_test_2, weights) + biases)
  

In [172]:
batch_size = 128
l2_weight =0.0


graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  #placeholders
  
  # Variables.
  weights_hidden = tf.Variable(tf.truncated_normal([image_size*image_size,256],stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases_hidden = tf.Variable(tf.zeros([256]))
  weights_hidden_2 = tf.Variable(tf.truncated_normal([256,256],stddev=np.sqrt(2.0 / 256)))
  biases_hidden_2 = tf.Variable(tf.zeros([256]))
  weights = tf.Variable(tf.truncated_normal([256, num_labels],stddev=np.sqrt(2.0 / 256)))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  hidden = tf.nn.relu(tf.matmul(tf_train_dataset,weights_hidden)+biases_hidden)
  hidden2 = tf.nn.relu(tf.matmul(hidden,weights_hidden_2)+biases_hidden_2) 
  logits = tf.matmul(hidden2, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + l2_weight*tf.nn.l2_loss(weights_hidden)
  
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000,0.65)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  hidden_valid = tf.nn.relu(tf.matmul(tf_valid_dataset,weights_hidden)+biases_hidden)
  hidden_valid_2 = tf.nn.relu(tf.matmul(hidden_valid,weights_hidden_2)+biases_hidden_2)
  valid_prediction = tf.nn.softmax(tf.matmul(hidden_valid_2, weights) + biases)
  hidden_test = tf.nn.relu(tf.matmul(tf_test_dataset,weights_hidden)+biases_hidden)
  hidden_test_2 = tf.nn.relu(tf.matmul(hidden_test,weights_hidden_2)+biases_hidden_2)
  test_prediction = tf.nn.softmax(tf.matmul(hidden_test_2, weights) + biases)

In [173]:
num_steps = 3001
t0 = time()

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step %500 == 0):
      print("##################################")
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
      print("time taken: %0.3fs"%(time()-t0))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
##################################
Minibatch loss at step 0: 2.302585
Minibatch accuracy: 7.0%
Validation accuracy: 10.0%
time taken: 0.939s
##################################
Minibatch loss at step 500: 2.311544
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%
time taken: 5.016s
##################################
Minibatch loss at step 1000: 2.302610
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
time taken: 9.007s
##################################
Minibatch loss at step 1500: 2.305011
Minibatch accuracy: 11.7%
Validation accuracy: 10.0%
time taken: 13.940s


KeyboardInterrupt: 

In [161]:

batch_size = 128
num_hidden_nodes1 = 1024
num_hidden_nodes2 = 100
beta_regul = 1e-3

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  global_step = tf.Variable(0)

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal(
        [image_size * image_size, num_hidden_nodes1],
        stddev=np.sqrt(2.0 / (image_size * image_size)))
    )
  biases1 = tf.Variable(tf.zeros([num_hidden_nodes1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes1, num_hidden_nodes2], stddev=np.sqrt(2.0 / num_hidden_nodes1)))
  biases2 = tf.Variable(tf.zeros([num_hidden_nodes2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_hidden_nodes2, num_labels], stddev=np.sqrt(2.0 / num_hidden_nodes2)))
  biases3 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  lay1_train = tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1)
  lay2_train = tf.nn.relu(tf.matmul(lay1_train, weights2) + biases2)
  logits = tf.matmul(lay2_train, weights3) + biases3
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels= tf_train_labels)) + \
      beta_regul * (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 1000, 0.65, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  lay1_valid = tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1)
  lay2_valid = tf.nn.relu(tf.matmul(lay1_valid, weights2) + biases2)
  valid_prediction = tf.nn.softmax(tf.matmul(lay2_valid, weights3) + biases3)
  lay1_test = tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1)
  lay2_test = tf.nn.relu(tf.matmul(lay1_test, weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(lay2_test, weights3) + biases3)

In [162]:
num_steps = 201

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 7.294692
Minibatch accuracy: 8.6%
Validation accuracy: 16.0%
Minibatch loss at step 50: 1275375104.000000
Minibatch accuracy: 10.2%
Validation accuracy: 10.0%
Minibatch loss at step 100: 1213159168.000000
Minibatch accuracy: 12.5%
Validation accuracy: 10.0%
Minibatch loss at step 150: 1153978368.000000
Minibatch accuracy: 9.4%
Validation accuracy: 10.0%


KeyboardInterrupt: 